# Sheep Counting Using Object Detection and ByteTrack
## **Line zone**

## 1. Model Loading and Initialization:

In [1]:
import os
import numpy as np
import supervision as sv
from ultralytics import YOLO

### Define directories

In [ ]:
HOME = os.getcwd()
print(HOME)

In [ ]:
import os
VIDEO_PATH = "../../data/Videos"
print(VIDEO_PATH)

### Load video file

In [ ]:
FILE = "sheep_crossing.MP4"
SOURCE_VIDEO_PATH = f"{VIDEO_PATH}/{FILE}"

In [ ]:
print(SOURCE_VIDEO_PATH)

In [ ]:
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
print(video_info)

### Define export folder

In [7]:
EXPORT = 'path/to/export/folder'

### Load pre-trained model

In [ ]:
# choose the model to use
SHEEP_MODEL = "../../Models/02_sheep_detection_v1/sheep_v1.pt"
print(SHEEP_MODEL)

In [ ]:
# load YOLO model 
model = YOLO(SHEEP_MODEL)
model.fuse()

## 2. Video Processing and Object Detection:


In [ ]:
# detection of objects on the video and recording of a new video with object predictions according to the YOLO model used
%cd {HOME}
!yolo task=detect mode=predict model="path/to/detector.pt" conf=0.50 source={SOURCE_VIDEO_PATH}

## 3. Integrating ByteTrack for Object Tracking and Sheep Counting Using a Virtual Line:


Create a video generator for our sample input file and display its first frame on the screen

In [9]:
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

In [ ]:
frame = next(generator)

sv.plot_image(frame, (12, 12))

### Define the position of the virtual line

In [ ]:
# line drawing
START = sv.Point(0, 850)
END = sv.Point(1440, 850)

# define line zone
line_zone = sv.LineZone(start=START, end=END)

line_zone_annotator = sv.LineZoneAnnotator(
    thickness=4,
    text_thickness=4,
    text_scale=2,
    custom_in_text = "in",
    custom_out_text = "out"
    )

annotated_frame = frame.copy()
annotated_frame = line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# show line annotator on one frame
sv.plot_image(annotated_frame, (12, 12))

#### Define tracking and detection methods

In [12]:
tracker = sv.ByteTrack(
    track_buffer=5,
    track_thresh=0.6, # default = 0.6
    match_thresh=0.8, # default = 0.8
    frame_rate=video_info.fps
)

### Improve vizualization

In [ ]:
color = sv.ColorPalette.DEFAULT
color_type = sv.ColorLookup.TRACK

# add annotators
bounding_box_annotator = sv.BoundingBoxAnnotator(thickness=4, color = color, color_lookup=color_type)
label_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=1)
trace_annotator = sv.TraceAnnotator(thickness=4)

def callback(frame: np.ndarray) -> sv.Detections:
    result = model(frame)[0]
    detection_model = sv.Detections.from_ultralytics(result)
    detections = tracker.update_with_detections(detection_model)

    # define labels to show on each bounding box
    labels = [
        f'#{tracker_id} {confidence:0.2f}'
        for tracker_id, confidence 
        in zip(detections.tracker_id, detections.confidence)
    ]

    # annotation on frame 
    annotated_frame = bounding_box_annotator.annotate(
        scene = frame.copy(),
        detections = detections)

    annotated_frame = label_annotator.annotate(
        annotated_frame, 
        detections, 
        labels)
    
    annotated_frame = trace_annotator.annotate(
        annotated_frame,
        detections= detections
    )

    # update in_count and out_count based on objects that cross the line
    line_zone.trigger(detections)

    return line_zone_annotator.annotate(annotated_frame, line_counter = line_zone)

# show the results on one frame
test_annotated_frame = callback(annotated_frame)
sv.plot_image(test_annotated_frame, (12, 12))

#### Process video

In [ ]:
# choose the output repository and output file name
MAIN_OUTPUT_PATH = EXPORT + f"/name_of_output_file"
print(MAIN_OUTPUT_PATH)

In [ ]:
# initiate frames generator
frames_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)

# process video
with sv.VideoSink(target_path=MAIN_OUTPUT_PATH, 
                  video_info=video_info) as sink:
  heatmap = None
  for i, frame in enumerate(frames_generator):
    annotated_frame = callback(frame)
    
    # to visualize tracking and counting frame by frame
    #sv.plot_image(annotated_frame)

    # Send as frame to video
    sink.write_frame(frame=annotated_frame)